<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import mediapipe as mp
import numpy as np
import cv2
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
import pandas as pd
from tqdm import tqdm
import gc
from operator import add

2023-08-04 14:20:42.862149: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-04 14:20:51.686727: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [4]:
cap = cv2.VideoCapture("file://media/kristian/HDD/ASL_Citizen/ASL_Citizen/videos/0023159093449660606-CAFETERIA.mp4")
#cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(static_image_mode=False,
                          model_complexity=2) as holistic:
    while True:
        ret, frame = cap.read()
        if ret == False:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)
        # face
        mp_drawing.draw_landmarks(
            frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
            mp_drawing.DrawingSpec(color=(0, 255, 255),
                                   thickness=1,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 128, 255), thickness=2))

        # Left hand
        mp_drawing.draw_landmarks(
            frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Right Hand
        mp_drawing.draw_landmarks(
            frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Posture
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))
        frame = cv2.flip(frame, 1)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()

In [5]:
results

mediapipe.python.solution_base.SolutionOutputs

In [6]:
len(results.pose_landmarks.landmark)

33

In [7]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [ ]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [8]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [9]:
result_test = extract_keypoints(results)

In [10]:
result_test

array([ 0.53414291,  0.4640485 , -0.59761614, ...,  0.        ,
        0.        ,  0.        ])

In [11]:
def signvideodataframe(filepath):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(filepath)

    # Define the path to the videos directory
    path = 'file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/videos/'

    # Create a 'Path' column by concatenating the path with 'Video file' column
    df['Path'] = path + df['Video file']
    
#    df['Gloss'] = df['Gloss'].str.replace('\d+', '',regex=True)

    # Calculate the frequency of each gloss and create a 'frequency' column
    df['frequency'] = df['Gloss'].map(df['Gloss'].value_counts())

    # Sort the DataFrame by the 'Gloss' column
    df = df.sort_values(by='Gloss')

    # Reset the index of the DataFrame
    df = df.reset_index(drop=True)

    return df

In [12]:
traindf = signvideodataframe('file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/splits/train.csv')

In [13]:
traindf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,frequency
0,P31,3827306090663467-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
1,P37,16792698524451422-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
2,P11,6868778695018762-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
3,P11,6870709051348651-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
4,P50,0719792557216079-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15


In [14]:
traindf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,frequency
40149,P37,9716493262876276-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
40150,P31,7550572181460327-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
40151,P46,47985881750082227-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
40152,P50,04671245574824856-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
40153,P51,19959052532136146-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15


In [15]:
traindf['Path'][0]

'file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/videos/3827306090663467-1 DOLLAR.mp4'

In [16]:
cap = cv2.VideoCapture(traindf['Path'][0])
#cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(static_image_mode=False,
                          model_complexity=2) as holistic:
    while True:
        ret, frame = cap.read()
        if ret == False:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)
        # face
        mp_drawing.draw_landmarks(
            frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
            mp_drawing.DrawingSpec(color=(0, 255, 255),
                                   thickness=1,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 128, 255), thickness=2))

        # Left hand
        mp_drawing.draw_landmarks(
            frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Right Hand
        mp_drawing.draw_landmarks(
            frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Posture
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))
        frame = cv2.flip(frame, 1)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()

[ERROR:0@112.167] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /io/opencv/modules/videoio/src/cap_images.cpp:267: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'




In [17]:
traindf.sample(20)

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,frequency
28637,P31,4135993368940054-RIDE 1.mp4,RIDE2,F_03_023,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,14
36691,P33,6060721040022268-TRIANGLE.mp4,TRIANGLE,J_02_080,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,16
11247,P40,355641287689366-ELEVATOR.mp4,ELEVATOR1,G_03_036,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19
1438,P43,20257590002949333-ATLANTA.mp4,ATLANTA,K_03_107,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
8672,P52,6472978554779458-seedDEAF SCHOOL.mp4,DEAFSCHOOL,H_01_022,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
10606,P52,38321646441276225-seedDRY.mp4,DRY,F_03_005,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
6393,P30,958412058407474-CHOP.mp4,CHOP,F_02_099,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,16
6187,P40,5685505516650793-CHERRY.mp4,CHERRY2,C_01_083,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,14
28651,P16,24705661999001016-RIGHT 1.mp4,RIGHT1,C_01_065,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
15501,P51,9784115170189687-GROUP.mp4,GROUP,J_01_001,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15


In [18]:
traindf = traindf[traindf['frequency'] > 17]
traindf = traindf.reset_index(drop=True)

In [21]:
traindf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,frequency
0,P52,07157565148825373-seedAXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19
1,P28,7179300005186042-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19
2,P29,16216064841959765-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19
3,P37,6193814382865199-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19
4,P40,5947453960317015-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19


In [22]:
traindf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,frequency
945,P50,9161417844146778-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,20
946,P14,00930662603221255-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,20
947,P27,82063651021682-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,20
948,P40,5268072837528903-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,20
949,P40,6363286086951516-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,20


In [23]:
traindf.describe()

,frequency
count,950.000000
mean,19.082105
std,1.316026
min,18.000000
25%,18.000000
50%,19.000000
75%,20.000000
max,24.000000


In [24]:
#traindf['Gloss'] = traindf['Gloss'].str.replace('\d+', '',regex=True)

In [25]:
traindf['Gloss'].nunique()

50

In [26]:
words = traindf['Gloss'].unique()
words

array(['AXE1', 'BACKPACK1', 'BASKETBALL1', 'BEE1', 'BELT1', 'BITE1',
       'BREAKFAST1', 'CANCER1', 'CHRISTMAS1', 'CONFUSED1', 'DARK1',
       'DEAF1', 'DECIDE1', 'DEMAND1', 'DEVELOP1', 'DINNER1', 'DOG1',
       'DOWNSIZE1', 'DRAG1', 'EAT1', 'EDIT1', 'ELEVATOR1', 'FINE1',
       'FLOAT1', 'FOREIGNER1', 'GUESS1', 'HALLOWEEN1', 'HOSPITAL1',
       'HURDLE/TRIP1', 'JEWELRY', 'KNIGHT1', 'LOCK1', 'LUNCH1', 'MAPLE',
       'MEAT1', 'MECHANIC1', 'MICROSCOPE1', 'MOVIE1', 'NOON1', 'PARTY1',
       'PATIENT2', 'RAZOR2', 'RIVER1', 'ROCKINGCHAIR1', 'SHAVE1', 'SINK',
       'SQUEEZE', 'THEY1', 'TWINS1', 'WHATFOR1'], dtype=object)

In [27]:
valdf = signvideodataframe('file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/splits/val.csv') 

In [28]:
valdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,frequency
0,P26,22595012150860327-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3
1,P39,7421622940519235-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3
2,P21,686738356933241-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3
3,P12,9219095671540121-5 DOLLARS.mp4,5DOLLARS,B_01_062,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
4,P21,1448188216215387-5 DOLLARS.mp4,5DOLLARS,B_01_062,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4


In [29]:
valdf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,frequency
10299,P21,6959326205750493-ZOOM IN.mp4,ZOOMIN,B_02_056,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3
10300,P12,7758716133684984-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
10301,P39,844134294032034-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
10302,P21,5548062993721732-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
10303,P26,19366754134806952-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4


In [30]:
testdf = signvideodataframe('file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/splits/test.csv')

In [31]:
testdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,frequency
0,P18,23521769221811684-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
1,P42,023931338852502426-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
2,P49,4893817008748198-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
3,P17,13991818149960333-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
4,P49,34625615110480457-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13


In [32]:
testdf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,frequency
32936,P18,4320702510886756-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
32937,P9,7676354653247301-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
32938,P47,5386272465310649-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
32939,P18,738440364224181-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
32940,P17,9953298353288469-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13


In [33]:
valdf = valdf[valdf['Gloss'].isin(words)]
valdf = valdf.reset_index(drop=True)

In [34]:
valdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,frequency
0,P39,19778675091674147-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
1,P26,8581142177964065-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
2,P39,3877478645046861-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
3,P21,8521417940364975-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
4,P5,9990244640190733-BACKPACK.mp4,BACKPACK1,G_03_091,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3


In [35]:
valdf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,frequency
186,P26,032677896012150764-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5
187,P5,6523145816470133-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5
188,P39,0283886564670357-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5
189,P21,6142521746642153-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5
190,P12,06408604416165864-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5


In [36]:
valdf['Gloss'].nunique()

50

In [37]:
valdf.describe()

,frequency
count,191.000000
mean,3.963351
std,0.770316
min,3.000000
25%,3.000000
50%,4.000000
75%,4.000000
max,6.000000


In [38]:
valdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,frequency
0,P39,19778675091674147-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
1,P26,8581142177964065-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
2,P39,3877478645046861-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
3,P21,8521417940364975-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
4,P5,9990244640190733-BACKPACK.mp4,BACKPACK1,G_03_091,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3


In [39]:
testdf = testdf[testdf['Gloss'].isin(words)]
testdf = testdf.reset_index(drop=True)

In [40]:
testdf['Gloss'].nunique()

50

In [41]:
testdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,frequency
0,P15,33721516025652254-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
1,P47,519067006979435-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
2,P35,5104381603195376-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
3,P42,44458614013793873-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
4,P42,9604797909481075-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15


In [42]:
testdf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,frequency
724,P42,3448123355846451-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18
725,P47,034046510887022485-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18
726,P22,9359050586202402-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18
727,P42,2748457214117681-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18
728,P15,540014801916062-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18


In [43]:
testdf.describe()

,frequency
count,729.000000
mean,14.978052
std,2.229192
min,9.000000
25%,14.000000
50%,15.000000
75%,16.000000
max,20.000000


In [44]:
traindf['Gloss'] = traindf['Gloss'].str.replace('\d+', '',regex=True)
valdf['Gloss'] = valdf['Gloss'].str.replace('\d+', '',regex=True)
testdf['Gloss'] = testdf['Gloss'].str.replace('\d+', '',regex=True)

In [45]:
words = traindf['Gloss'].unique()
words

array(['AXE', 'BACKPACK', 'BASKETBALL', 'BEE', 'BELT', 'BITE',
       'BREAKFAST', 'CANCER', 'CHRISTMAS', 'CONFUSED', 'DARK', 'DEAF',
       'DECIDE', 'DEMAND', 'DEVELOP', 'DINNER', 'DOG', 'DOWNSIZE', 'DRAG',
       'EAT', 'EDIT', 'ELEVATOR', 'FINE', 'FLOAT', 'FOREIGNER', 'GUESS',
       'HALLOWEEN', 'HOSPITAL', 'HURDLE/TRIP', 'JEWELRY', 'KNIGHT',
       'LOCK', 'LUNCH', 'MAPLE', 'MEAT', 'MECHANIC', 'MICROSCOPE',
       'MOVIE', 'NOON', 'PARTY', 'PATIENT', 'RAZOR', 'RIVER',
       'ROCKINGCHAIR', 'SHAVE', 'SINK', 'SQUEEZE', 'THEY', 'TWINS',
       'WHATFOR'], dtype=object)

In [46]:
words = valdf['Gloss'].unique()
words

array(['AXE', 'BACKPACK', 'BASKETBALL', 'BEE', 'BELT', 'BITE',
       'BREAKFAST', 'CANCER', 'CHRISTMAS', 'CONFUSED', 'DARK', 'DEAF',
       'DECIDE', 'DEMAND', 'DEVELOP', 'DINNER', 'DOG', 'DOWNSIZE', 'DRAG',
       'EAT', 'EDIT', 'ELEVATOR', 'FINE', 'FLOAT', 'FOREIGNER', 'GUESS',
       'HALLOWEEN', 'HOSPITAL', 'HURDLE/TRIP', 'JEWELRY', 'KNIGHT',
       'LOCK', 'LUNCH', 'MAPLE', 'MEAT', 'MECHANIC', 'MICROSCOPE',
       'MOVIE', 'NOON', 'PARTY', 'PATIENT', 'RAZOR', 'RIVER',
       'ROCKINGCHAIR', 'SHAVE', 'SINK', 'SQUEEZE', 'THEY', 'TWINS',
       'WHATFOR'], dtype=object)

In [47]:
words = testdf['Gloss'].unique()
words

array(['AXE', 'BACKPACK', 'BASKETBALL', 'BEE', 'BELT', 'BITE',
       'BREAKFAST', 'CANCER', 'CHRISTMAS', 'CONFUSED', 'DARK', 'DEAF',
       'DECIDE', 'DEMAND', 'DEVELOP', 'DINNER', 'DOG', 'DOWNSIZE', 'DRAG',
       'EAT', 'EDIT', 'ELEVATOR', 'FINE', 'FLOAT', 'FOREIGNER', 'GUESS',
       'HALLOWEEN', 'HOSPITAL', 'HURDLE/TRIP', 'JEWELRY', 'KNIGHT',
       'LOCK', 'LUNCH', 'MAPLE', 'MEAT', 'MECHANIC', 'MICROSCOPE',
       'MOVIE', 'NOON', 'PARTY', 'PATIENT', 'RAZOR', 'RIVER',
       'ROCKINGCHAIR', 'SHAVE', 'SINK', 'SQUEEZE', 'THEY', 'TWINS',
       'WHATFOR'], dtype=object)

In [48]:
print(traindf['Gloss'].nunique())
print(valdf['Gloss'].nunique())
print(testdf['Gloss'].nunique())

50
50
50


In [52]:
cap = cv2.VideoCapture(traindf.Path[643])
#cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

with mp_holistic.Holistic(static_image_mode=False,
                          model_complexity=2) as holistic:
    while True:
        ret, frame = cap.read()
        if ret == False:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)
        # rostro
        mp_drawing.draw_landmarks(
            frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
            mp_drawing.DrawingSpec(color=(0, 255, 255),
                                   thickness=1,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 128, 255), thickness=2))

        # Mano izquieda (azul)
        mp_drawing.draw_landmarks(
            frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Mano derecha (verde)
        mp_drawing.draw_landmarks(
            frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Postura
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))
        frame = cv2.flip(frame, 1)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()

In [53]:
traindf.Path[643]

'file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/videos/8626819778158101-MAPLE.mp4'

In [54]:
# Define a function to extract keypoints from a single video file
def extract_keypoints_from_file(filepath):
    # Initialize a MediaPipe Holistic model
    holistic = mp.solutions.holistic.Holistic(static_image_mode=False,
                                              model_complexity=0,
                                              min_detection_confidence=0.5,
                                              min_tracking_confidence=0.5)

    # Open the video file
    cap = cv2.VideoCapture(filepath)

    # Initialize an empty list to store the keypoints for each frame
    keypoints = []

    # Loop through the frames of the video
    while cap.isOpened():
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB color space
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Use the Holistic model to detect landmarks for the face, pose, and hands
        results = holistic.process(frame)

        # Extract the keypoints from the results object and append them to the keypoints list
        keypoints.append(extract_keypoints(results))

    # Release the video capture object and the Holistic model
    cap.release()
    holistic.close()

    return np.array(keypoints)

In [55]:
extract_keypoints_from_file(traindf.Path[643]).shape

(83, 1662)

In [467]:
cap = cv2.VideoCapture(traindf.Path[643])
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print( length )

83


In [468]:
type(extract_keypoints_from_file(traindf.Path[643]))

numpy.ndarray

In [469]:
traindf.count()

Participant ID    950
Video file        950
Gloss             950
ASL-LEX Code      950
Path              950
frequency         950
dtype: int64

In [470]:
def frames_from_file(filepath):
    cap = cv2.VideoCapture(filepath)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    return length

In [471]:
frames_from_file(traindf.Path[643])

83

In [297]:
tqdm.pandas()
valdf['Frames'] = valdf['Path'].progress_apply(frames_from_file)

  1%|█▌                                                                                                                                                       | 2/191 [00:02<03:44,  1.19s/it][ERROR:0@11098.891] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /io/opencv/modules/videoio/src/cap_images.cpp:267: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'


[ERROR:0@11099.972] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /io/opencv/modules/videoio/src/cap_images.cpp:267: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'


  2%|███▏                                                                                                                                                     | 4/191 [00:03<02:31,  1.24it/s][ERROR:0@11101.032] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /io/opencv/modules/videoio/src/cap_i

[ERROR:0@11129.221] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /io/opencv/modules/videoio/src/cap_images.cpp:267: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'


 18%|███████████████████████████                                                                                                                             | 34/191 [00:32<02:01,  1.29it/s][ERROR:0@11129.222] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /io/opencv/modules/videoio/src/cap_images.cpp:267: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'


 19%|█████████████████████████████▍                                                                                                                          | 37/191 [00:40<03:52,  1.51s/it][ERROR:0@11137.326] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /io/opencv/modules/videoio/src/cap_i

 37%|███████████████████████████████████████████████████████▋                                                                                                | 70/191 [01:44<04:48,  2.39s/it][ERROR:0@11202.221] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /io/opencv/modules/videoio/src/cap_images.cpp:267: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'


 38%|█████████████████████████████████████████████████████████▎                                                                                              | 72/191 [01:50<05:14,  2.64s/it][ERROR:0@11206.577] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /io/opencv/modules/videoio/src/cap_images.cpp:267: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'


 39%|███████████████████████████████████████████████████████████▋                                                                           

[ERROR:0@11260.569] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /io/opencv/modules/videoio/src/cap_images.cpp:267: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'


 57%|█████████████████████████████████████████████████████████████████████████████████████▍                                                                 | 108/191 [02:50<02:31,  1.82s/it][ERROR:0@11267.199] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /io/opencv/modules/videoio/src/cap_images.cpp:267: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'


[ERROR:0@11267.200] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /io/opencv/modules/videoio/src/cap_images.cpp:267: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'


 58%|█████████████████████████████████████████████████████████████████████████████████████

 78%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                                 | 149/191 [04:15<01:01,  1.46s/it][ERROR:0@11353.037] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /io/opencv/modules/videoio/src/cap_images.cpp:267: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'


 79%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                | 150/191 [04:16<00:56,  1.39s/it][ERROR:0@11353.038] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /io/opencv/modules/videoio/src/cap_images.cpp:267: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'


 82%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎           

 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 190/191 [05:24<00:01,  1.34s/it][ERROR:0@11421.299] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /io/opencv/modules/videoio/src/cap_images.cpp:267: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'


[ERROR:0@11422.376] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /io/opencv/modules/videoio/src/cap_images.cpp:267: error: (-215:Assertion failed) number < max_number in function 'icvExtractPattern'


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 191/191 [05:25<00:00,  1.71s/it]


In [292]:
traindf['Frames'] = traindf['Frames'].astype('int')

In [293]:
traindf['Frames'].max()

337

In [291]:
traindf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,frequency,Frames
0,P52,07157565148825373-seedAXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19,100
1,P28,7179300005186042-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19,0
2,P29,16216064841959765-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19,0
3,P37,6193814382865199-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19,0
4,P40,5947453960317015-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19,0


In [129]:
keypoints_series = video_df['path'].progress_apply(extract_keypoints_from_file)

100%|████████████████████████████████████| 1405/1405 [2:56:25<00:00,  7.53s/it]


In [130]:
keypoints_series.shape

(1405,)

In [131]:
max(keypoints.shape[0] for keypoints in keypoints_series)

212

In [132]:
min(keypoints.shape[0] for keypoints in keypoints_series)


17

In [52]:
max_len = video_df['frames'].max()

In [298]:
valdf.describe()

,frequency,Frames
count,191.000000,191.000000
mean,3.963351,36.020942
std,0.770316,51.732608
min,3.000000,0.000000
25%,3.000000,0.000000
50%,4.000000,0.000000
75%,4.000000,64.500000
max,6.000000,170.000000


In [296]:
traindf.sample(50)

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,frequency,Frames
293,P37,7262160429010622-DINNER.mp4,DINNER1,G_03_044,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,77
930,P37,6237575353180616-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,20,0
895,P40,7717057217254395-THEY.mp4,THEY1,F_02_103,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,0
324,P29,5465905150307049-DOG.mp4,DOG1,A_01_056,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,24,0
768,P37,7753126798089989-PATIENT 2.mp4,PATIENT2,B_01_085,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19,56
727,P40,8667743378159272-NOON.mp4,NOON1,G_02_014,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19,0
440,P33,07214322840682375-FINE.mp4,FINE1,C_01_043,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19,0
119,P50,7717758625736204-BREAKFAST.mp4,BREAKFAST1,J_01_053,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,21,0
541,P29,02514324335585849-HURDLE-TRIP.mp4,HURDLE/TRIP1,G_01_036,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,22,0
534,P29,5903218158401913-HOSPITAL.mp4,HOSPITAL1,B_02_026,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,0


In [1]:
keypoints_np = np.zeros((len(keypoints_series), max_len, 1662))
for i, keypoints in enumerate(keypoints_series):
    keypoints_np[i, :keypoints.shape[0], :] = keypoints

NameError: name 'keypoints_series' is not defined

In [1]:
keypoints_np.shape

NameError: name 'keypoints_np' is not defined

In [137]:
np.save("/home/kristian/Capstone/keypoints/mediapipe_keypoints.npy", keypoints_np)

In [53]:
import numpy as np
keypoints_np=np.load('/home/kristian/Capstone/keypoints/mediapipe_keypoints.npy')

In [54]:
keypoints_np.shape

(1405, 212, 1662)

# Preprocessing

In [55]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from keras.utils import to_categorical
import numpy as np
from keras.preprocessing.sequence import pad_sequences

In [56]:
# Pad sequences with zeros and create a mask
padded_keypoints = pad_sequences(keypoints_np, maxlen=212, padding='post', value=0.0, dtype='float32')
mask = (padded_keypoints.sum(axis=2) != 0).astype('float32')  # Create a mask

In [57]:
mask.shape

(1405, 212)

In [58]:
traindf.head()

,path,target,file,originaltarget,frequency,frames
0,file:/home/kristian/WLASL/videos_organized/acc...,accident,00618.mp4,accident,17,27
1,file:/home/kristian/WLASL/videos_organized/acc...,accident,00623.mp4,accident,17,104
2,file:/home/kristian/WLASL/videos_organized/acc...,accident,00624.mp4,accident,17,109
3,file:/home/kristian/WLASL/videos_organized/acc...,accident,00625.mp4,accident,17,34
4,file:/home/kristian/WLASL/videos_organized/acc...,accident,00626.mp4,accident,17,44


In [59]:
encoder = OrdinalEncoder()

In [60]:
video_df['cat_label'] = encoder.fit_transform(video_df[['target']]).astype(int)

In [61]:
video_df.head()

,path,target,file,originaltarget,frequency,frames,cat_label
0,file:/home/kristian/WLASL/videos_organized/acc...,accident,00618.mp4,accident,17,27,0
1,file:/home/kristian/WLASL/videos_organized/acc...,accident,00623.mp4,accident,17,104,0
2,file:/home/kristian/WLASL/videos_organized/acc...,accident,00624.mp4,accident,17,109,0
3,file:/home/kristian/WLASL/videos_organized/acc...,accident,00625.mp4,accident,17,34,0
4,file:/home/kristian/WLASL/videos_organized/acc...,accident,00626.mp4,accident,17,44,0


In [62]:
video_df.tail()

,path,target,file,originaltarget,frequency,frames,cat_label
1400,file:/home/kristian/WLASL/videos_organized/yes...,yesterday,64314.mp4,yesterday,15,64,83
1401,file:/home/kristian/WLASL/videos_organized/yes...,yesterday,64315.mp4,yesterday,15,109,83
1402,file:/home/kristian/WLASL/videos_organized/yes...,yesterday,66821.mp4,yesterday,15,64,83
1403,file:/home/kristian/WLASL/videos_organized/yes...,yesterday,66822.mp4,yesterday,15,62,83
1404,file:/home/kristian/WLASL/videos_organized/yes...,yesterday,70347.mp4,yesterday,15,111,83


In [63]:
label_map = {label:num for num, label in enumerate(video_df['target'].unique())}

In [64]:
label_map

{'accident': 0,
 'africa': 1,
 'all': 2,
 'apple': 3,
 'bed': 4,
 'before': 5,
 'bird': 6,
 'black': 7,
 'book': 8,
 'bowling': 9,
 'candy': 10,
 'chair': 11,
 'change': 12,
 'cheat': 13,
 'check': 14,
 'city': 15,
 'clothes': 16,
 'color': 17,
 'computer': 18,
 'convince': 19,
 'cook': 20,
 'cool': 21,
 'corn': 22,
 'cousin': 23,
 'cow': 24,
 'dark': 25,
 'deaf': 26,
 'decide': 27,
 'dog': 28,
 'drink': 29,
 'family': 30,
 'fine': 31,
 'finish': 32,
 'fish': 33,
 'forget': 34,
 'full': 35,
 'give': 36,
 'go': 37,
 'graduate': 38,
 'hair': 39,
 'hat': 40,
 'headache': 41,
 'hearing': 42,
 'help': 43,
 'hot': 44,
 'kiss': 45,
 'language': 46,
 'last': 47,
 'later': 48,
 'like': 49,
 'man': 50,
 'many': 51,
 'meet': 52,
 'mother': 53,
 'no': 54,
 'now': 55,
 'orange': 56,
 'paper': 57,
 'pink': 58,
 'pizza': 59,
 'play': 60,
 'pull': 61,
 'school': 62,
 'secretary': 63,
 'shirt': 64,
 'short': 65,
 'son': 66,
 'study': 67,
 'table': 68,
 'tall': 69,
 'thanksgiving': 70,
 'thin': 71,
 'th

In [65]:
video_df.sample(10)

,path,target,file,originaltarget,frequency,frames,cat_label
300,file:/home/kristian/WLASL/videos_organized/clo...,clothes,11321.mp4,clothes,23,139,16
610,file:/home/kristian/WLASL/videos_organized/for...,forget,68054.mp4,forget,16,51,34
1356,file:/home/kristian/WLASL/videos_organized/yea...,year,64201.mp4,year,16,77,81
1273,file:/home/kristian/WLASL/videos_organized/wat...,water,62479.mp4,water,15,53,76
733,file:/home/kristian/WLASL/videos_organized/hea...,hearing,26986.mp4,hearing,15,60,42
640,file:/home/kristian/WLASL/videos_organized/giv...,give,24660.mp4,give,15,70,36
206,file:/home/kristian/WLASL/videos_organized/cha...,chair,09848.mp4,chair,22,96,11
1216,file:/home/kristian/WLASL/videos_organized/thu...,thursday,58365.mp4,thursday,16,69,72
1315,file:/home/kristian/WLASL/videos_organized/who...,who,63237.mp4,who,17,47,78
1095,file:/home/kristian/WLASL/videos_organized/sho...,short,51221.mp4,short,16,100,65


In [66]:
Y = encoder.fit_transform(video_df[['target']])

In [67]:
Y

array([[ 0.],
       [ 0.],
       [ 0.],
       ...,
       [83.],
       [83.],
       [83.]])

In [70]:
n_classes = 84
print("Shape before one-hot encoding: ", Y.shape)
Y = to_categorical(Y, n_classes)
print("Shape after one-hot encoding: ", Y.shape)

Shape before one-hot encoding:  (1405, 1)
Shape after one-hot encoding:  (1405, 84)


In [68]:
X = keypoints_np

In [69]:
X.shape

(1405, 212, 1662)

In [71]:
np.save("/home/kristian/Capstone/keypoints/labels_array_1D.npy", Y)

In [7]:
Y=np.load('/home/kristian/Capstone/keypoints/labels_array.npy')

In [8]:
type(Y)

numpy.ndarray

In [9]:
Y.shape

(1405, 84)

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.2,
                                                    random_state=10)

In [13]:
from sklearn.preprocessing import MinMaxScaler

# Reshape your input data to match the shape (samples, features)
# Assuming your data is named "input_data"
num_samples, num_frames, num_keypoints = X.shape
input_data_reshaped = X.reshape(num_samples, -1)

# Create a MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the data
input_data_scaled = scaler.fit_transform(input_data_reshaped)

# Reshape the scaled data back to the original shape
input_data_scaled = input_data_scaled.reshape(num_samples, num_frames, num_keypoints)

In [12]:
input_data_scaled.shape

(1405, 212, 1662)

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(input_data_scaled,
                                                    Y,
                                                    test_size=0.,
                                                    random_state=10)

NameError: name 'input_data_scaled' is not defined

In [21]:
mask_train.shape

(1124, 212)

In [23]:
X_train.shape

(1264, 212, 1662)

# Model

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam


In [12]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)

In [14]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [16]:
model = Sequential(name='sequential')

2023-08-03 17:33:23.873474: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1739 MB memory:  -> device: 0, name: NVIDIA GeForce MX250, pci bus id: 0000:01:00.0, compute capability: 6.1


In [17]:
model.add(LSTM(128, return_sequences=True, activation='relu', dropout=0.5,input_shape=(212,1662)))
#model.add(Dropout(0.5))
model.add(LSTM(64, return_sequences=True, activation='sigmoid',dropout=.5))
#model.add(Dropout(0.25))
model.add(LSTM(128, return_sequences=False, activation='sigmoid'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(Y.shape[1], activation='softmax'))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 212, 128)          916992    
                                                                 
 lstm_1 (LSTM)               (None, 212, 64)           49408     
                                                                 
 lstm_2 (LSTM)               (None, 128)               98816     
                                                                 
 dense (Dense)               (None, 128)               16512     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 84)                5460      
                                                                 
Total params: 1095444 (4.18 MB)
Trainable params: 109544

In [19]:
optimizer = Adam(learning_rate=.001,clipvalue=0)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [20]:
model.fit(X_train, Y_train, epochs=20, callbacks=[tb_callback],batch_size =8)

2023-08-03 17:33:39.635109: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1781451264 exceeds 10% of free system memory.
2023-08-03 17:33:41.049569: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1781451264 exceeds 10% of free system memory.


Epoch 1/20


2023-08-03 17:33:53.017370: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 256.0KiB (rounded to 262144)requested by op Fill
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-08-03 17:33:53.017446: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-08-03 17:33:53.017473: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 26, Chunks in use: 26. 6.5KiB allocated for chunks. 6.5KiB in use in bin. 369B client-requested in use in bin.
2023-08-03 17:33:53.017494: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 2, Chunks in use: 2. 1.0KiB allocated for chunks. 1.0KiB in use in bin. 848B client-requested in use in bin.
2023-08-03 17:33:53.017514: I tensorflow/tsl/framework/bfc_allocator.cc

ResourceExhaustedError: in user code:

    File "/home/kristian/miniconda3/lib/python3.10/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/home/kristian/miniconda3/lib/python3.10/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/kristian/miniconda3/lib/python3.10/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/home/kristian/miniconda3/lib/python3.10/site-packages/keras/src/engine/training.py", line 1084, in train_step
        self.optimizer.minimize(loss, self.trainable_variables, tape=tape)
    File "/home/kristian/miniconda3/lib/python3.10/site-packages/keras/src/optimizers/optimizer.py", line 544, in minimize
        self.apply_gradients(grads_and_vars)
    File "/home/kristian/miniconda3/lib/python3.10/site-pack

SyntaxError: invalid syntax (2932272115.py, line 1)

In [20]:
with tf.device('/cpu:0'):
    model.fit(X_train, Y_train, epochs=20, callbacks=[tb_callback],batch_size =64)


2023-08-03 15:13:51.601746: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1584138624 exceeds 10% of free system memory.


Epoch 1/20


2023-08-03 15:13:58.156817: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at xla_ops.cc:444 : INVALID_ARGUMENT: Trying to access resource Resource-17-at-0x9a1e600 (defined @ /home/kristian/miniconda3/lib/python3.10/site-packages/keras/src/engine/base_layer_utils.py:137) located in device /job:localhost/replica:0/task:0/device:GPU:0 from device /job:localhost/replica:0/task:0/device:CPU:0
 Cf. https://www.tensorflow.org/xla/known_issues#tfvariable_on_a_different_device
2023-08-03 15:13:58.156864: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at xla_ops.cc:444 : INVALID_ARGUMENT: Trying to access resource Resource-16-at-0x9a18cb0 (defined @ /home/kristian/miniconda3/lib/python3.10/site-packages/keras/src/engine/base_layer_utils.py:137) located in device /job:localhost/replica:0/task:0/device:GPU:0 from device /job:localhost/replica:0/task:0/device:CPU:0
 Cf. https://www.tensorflow.org/xla/known_issues#tfvariable_on_a_different_device
2023-08-03 15:

InvalidArgumentError: Graph execution error:

Detected at node 'Adam/StatefulPartitionedCall_14' defined at (most recent call last):
    File "/home/kristian/miniconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/kristian/miniconda3/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/kristian/miniconda3/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/kristian/miniconda3/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/home/kristian/miniconda3/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/home/kristian/miniconda3/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/home/kristian/minicond

In [39]:
model = Sequential(name='sequential')

In [40]:
model.add(GRU(64, return_sequences=True, activation='tanh', input_shape=(212, 1662)))
model.add(GRU(128, return_sequences=True, activation='tanh'))
model.add(GRU(64, return_sequences=False, activation='tanh'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(Y.shape[1], activation='softmax'))

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_11 (GRU)                (None, 212, 64)           331776    
                                                                 
 gru_12 (GRU)                (None, 212, 128)          74496     
                                                                 
 gru_13 (GRU)                (None, 64)                37248     
                                                                 
 dense_15 (Dense)            (None, 64)                4160      
                                                                 
 dense_16 (Dense)            (None, 32)                2080      
                                                                 
 dense_17 (Dense)            (None, 84)                2772      
                                                                 
Total params: 452532 (1.73 MB)
Trainable params: 452532 

In [42]:
optimizer = Adam(learning_rate=.01,clipvalue=1.0)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [43]:
model.fit(X_train, Y_train, epochs=20, callbacks=[tb_callback],batch_size =16)

2023-08-03 14:22:57.722675: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1584138624 exceeds 10% of free system memory.
2023-08-03 14:22:59.241294: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1584138624 exceeds 10% of free system memory.


Epoch 1/20
71/71 [==============================] - 10s 49ms/step - loss: 4.4668 - categorical_accuracy: 0.0089
Epoch 2/20
71/71 [==============================] - 4s 50ms/step - loss: 4.4306 - categorical_accuracy: 0.0267
Epoch 3/20
71/71 [==============================] - 4s 50ms/step - loss: 4.4264 - categorical_accuracy: 0.0258
Epoch 4/20
71/71 [==============================] - 4s 50ms/step - loss: nan - categorical_accuracy: 0.0133
Epoch 5/20
71/71 [==============================] - 4s 51ms/step - loss: nan - categorical_accuracy: 0.0125
Epoch 6/20
71/71 [==============================] - 4s 51ms/step - loss: nan - categorical_accuracy: 0.0125
Epoch 7/20
71/71 [==============================] - 4s 51ms/step - loss: nan - categorical_accuracy: 0.0125
Epoch 8/20
 7/71 [=>............................] - ETA: 3s - loss: nan - categorical_accuracy: 0.0000e+00

KeyboardInterrupt: 

In [14]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, SpatialDropout1D

# Assuming you have your padded keypoints data (keypoints_np) and labels (labels)
# keypoints_np.shape = (num_samples, max_len, num_keypoints)
# labels.shape = (num_samples, num_classes)

# Define the CNN model
model = Sequential(name='sequential')

# Add convolutional layers
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(212, 1662),
                 kernel_regularizer=regularizers.l2(l=0.01)))
model.add(MaxPooling1D(pool_size=2))
#model.add(Conv1D(filters=64, kernel_size=3, activation='relu', kernel_regularizer=regularizers.l2(l=0.01)))
#model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', kernel_regularizer=regularizers.l2(l=0.01)))
model.add(MaxPooling1D(pool_size=2))

# droput layer, remove if no work
model.add(SpatialDropout1D(0.5))

# Flatten the output for the fully connected layers
model.add(Flatten())

# Add fully connected layers
model.add(Dense(128, activation='relu'))
#model.add(Dense(64, activation='relu'))
model.add(Dense(84, activation='softmax'))  # Output layer

optimizer = Adam(learning_rate=.0005)
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model
model.fit(X_train, Y_train, epochs=80, batch_size=64,validation_split=.3)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 210, 32)           159584    
                                                                 
 max_pooling1d_3 (MaxPoolin  (None, 105, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_4 (Conv1D)           (None, 103, 128)          12416     
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 51, 128)           0         
 g1D)                                                            
                                                                 
 spatial_dropout1d_1 (Spati  (None, 51, 128)           0         
 alDropout1D)                                                    
                                                        

2023-08-03 19:57:08.520945: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.03GiB (rounded to 1107769600)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-08-03 19:57:08.521032: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2023-08-03 19:57:08.521064: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 54, Chunks in use: 54. 13.5KiB allocated for chunks. 13.5KiB in use in bin. 2.5KiB client-requested in use in bin.
2023-08-03 19:57:08.521086: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 12, Chunks in use: 12. 6.0KiB allocated for chunks. 6.0KiB in use in bin. 5.3KiB client-requested in use in bin.
2023-08-03 19:57:08.521105: I tensorflow/tsl/framewor

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

ext 43
2023-08-03 19:57:08.522500: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7efca729bd00 of size 98304 next 49
2023-08-03 19:57:08.522512: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7efca72b3d00 of size 1572864 next 52
2023-08-03 19:57:08.522524: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7efca7433d00 of size 1884928 next 18446744073709551615
2023-08-03 19:57:08.522537: I tensorflow/tsl/framework/bfc_allocator.cc:1075] Next region of size 8388608
2023-08-03 19:57:08.522549: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7efca7600000 of size 1572864 next 25
2023-08-03 19:57:08.522561: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7efca7780000 of size 32768 next 55
2023-08-03 19:57:08.522572: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7efca7788000 of size 32768 next 56
2023-08-03 19:57:08.522584: I tensorflow/tsl/framework/bfc_allocator.cc:1095] InUse at 7efca7790000 of size 21504 next 59
2023-08-0

In [53]:
np.argmax(model.predict(X_test[[1]]))

1/1 [==============================] - 0s 25ms/step


18

In [52]:
np.argmax(Y_test[1])

41

In [38]:
model.predict(X_test[[50]])

1/1 [==============================] - 0s 20ms/step


array([[1.0129494e-06, 7.4129015e-17, 4.8822547e-11, 1.1211572e-13,
        5.1141450e-07, 1.4339512e-10, 6.1791358e-14, 5.4054793e-20,
        5.5316541e-15, 4.7033727e-12, 4.9788362e-10, 5.2938990e-11,
        2.1888662e-08, 8.2826823e-13, 3.0033429e-10, 2.5744862e-07,
        3.9990834e-07, 2.3802282e-19, 1.1033153e-15, 2.3090003e-07,
        6.4198423e-15, 5.0172403e-08, 8.0218030e-17, 8.2776613e-35,
        2.8937308e-20, 1.9814779e-19, 2.5020405e-14, 4.0026234e-17,
        2.3025483e-11, 1.3046385e-15, 1.1999841e-10, 2.4870423e-15,
        1.5261118e-06, 1.1967564e-05, 1.1540307e-22, 7.1786820e-07,
        2.3164215e-13, 3.3356621e-08, 7.5515008e-09, 2.1571792e-24,
        1.1177559e-24, 9.9436084e-11, 8.9361262e-28, 1.3001557e-17,
        2.2173275e-07, 1.4378391e-19, 4.7059345e-14, 5.1826217e-11,
        1.4783451e-07, 5.9562458e-19, 1.3092859e-10, 2.0330034e-15,
        6.0108329e-15, 9.5366354e-17, 5.7902678e-24, 6.8326376e-22,
        4.4553554e-13, 1.6354499e-19, 5.5277251e